In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import numba

In [2]:
import pandas as pd
import numpy as np
import quantstats as qs
qs.extend_pandas()

%matplotlib inline
import matplotlib.pyplot as plt
pd.options.display.max_rows = 99999

import warnings
warnings.filterwarnings("ignore")

def report(tradelist,initial_capital,type_analisy,Benchmark):
    %matplotlib inline
    import quantstats as qs
    qs.extend_pandas()

    tradelist = tradelist.fillna(0)
    tradelist = tradelist[tradelist!=0]
    tradelist.iloc[0] = tradelist.iloc[0]+initial_capital
    tradelist = tradelist.cumsum()
    
    if type_analisy == 0:
        qs.plots.snapshot(tradelist)
    if type_analisy == 1:    
        if len(Benchmark)>0:
            qs.reports.basic(tradelist,Benchmark)
        else:
            qs.reports.basic(tradelist)
    if type_analisy == 2:    
        if len(Benchmark)>0:
            qs.reports.full(tradelist,Benchmark)
        else:
            qs.reports.full(tradelist)
       
def remove_outlier(y):
    Q1 = np.percentile(y, 20, #25
                       interpolation = 'midpoint')
    Q3 = np.percentile(y, 80, #75
                       interpolation = 'midpoint')
    IQR = Q3 - Q1
    y[y>=Q3+1.5*IQR]=0
    y[y<=Q1-1.5*IQR]=0
    return y

def gross_profit(operations):
    return round(operations[operations > 0].sum(),2)
    
def gross_loss(operations):
    return round(operations[operations <= 0].sum(),2)
    
def profit_factor(operations):
    a = gross_profit(operations)
    b = gross_loss(operations)
    if b != 0:
        return round(abs(a / b), 2)
    else:
        return round(abs(a / 0.00000001), 2)
    
def kestner_ratio(operations):

    import numpy as np
    import matplotlib.pyplot as plt 
    from scipy import stats
    
    monthly_operations = operations.resample('D').sum().fillna(0)
    monthly_equity = monthly_operations.cumsum()
    index = np.array(np.arange(1,monthly_operations.count() + 1))
    
    x = index
    y = monthly_equity
    gradient, intercept, r_value, p_value, std_err = stats.linregress(x,y)
    
    if std_err != 0 and len(index) > 0:
        return round(gradient / (std_err * len(index)),5)
    else:
        return np.inf
    
def avg_trade(operations):
    return round(operations.mean(),2)

def check_position(path,commento):
    
    import pymt5adapter as mta
    from pymt5adapter.order import  Order
    mta.initialize(path)
    posizione_aperte = mta.positions_get()
    check = False
    for pos in posizione_aperte:
        if commento == pos.comment:
            check = True
    return check

In [3]:
%run ./Genetic_Lib.ipynb

In [8]:
def my_drawdown(series):
    equity = series.cumsum()
    maxvalue = equity.expanding(0).max()
    drawdown = equity - maxvalue
    drawdown_series = pd.Series(drawdown, index = equity.index)
    return drawdown_series

def my_max_draw_down(series):
    dd = my_drawdown(series)
    return abs(round(dd.min(),2))

def my_ret_dd(series):
    return round((series.sum()/abs(my_max_draw_down(series))),2)

def my_sharpe_ratio(series):
    equity = series.cumsum()
    netprofit = equity[-1]
    std = equity.std()
    if std != 0:
        return round(netprofit / std,2)
    else:
        return np.inf
    
def my_consec_win(series):
    data = series.to_frame()
    data[data>0]="TMP"
    data[data!="TMP"]=0
    data[data=="TMP"]=1
    data.columns=(["result"])
    data['start_of_streak'] = data['result'].ne(data['result'].shift())
    data['streak_id'] = data.start_of_streak.cumsum()
    data['streak_counter'] = data.groupby('streak_id').cumcount() + 1
    streaks = pd.concat([series, data['streak_counter']], axis=1)
    return streaks[streaks.iloc[:,0]>0].streak_counter.max()

def my_consec_loss(series):
    data = series.to_frame()
    data[data>0]="TMP"
    data[data!="TMP"]=0
    data[data=="TMP"]=1
    data.columns=(["result"])
    data['start_of_streak'] = data['result'].ne(data['result'].shift())
    data['streak_id'] = data.start_of_streak.cumsum()
    data['streak_counter'] = data.groupby('streak_id').cumcount() + 1
    streaks = pd.concat([series, data['streak_counter']], axis=1)
    return streaks[streaks.iloc[:,0]<0].streak_counter.max()

def my_mean_loss(series):
    return (round(series[series<0].dropna().mean(),2))*(-1)

def my_mean_win(series):
    return round(series[series>0].dropna().mean(),2)

def my_avg_trade(series):
    return round(series.mean(),2)

# carico df delle strategie 

In [1]:
####### da data_bk= dataframe del portafoglio o database da monitorare 

In [ ]:
metric_list = ["my_drawdown","my_max_draw_down" , "my_ret_dd" , "my_sharpe_ratio" , "my_consec_loss" ,"my_mean_loss","my_mean_win","my_avg_trade"]


   
strategy_ok = [] 
strategy_fail = []
data_check = pd.DataFrame()
    
for i in tqdm(data_bk.columns):

    real_money_date = ##### leggi la data dal csv di inizio real 

    df_comparations = data_bk[i].to_frame().copy()
    df_comparations = df_comparations[df_comparations!=0].dropna()

    tmp_plot = df_comparations.iloc[:,0].cumsum()
    tmp_date = len(tmp_plot.loc[:real_money_date])
    #pd.Series(tmp_plot.values).plot(title=i,figsize=(10,10))
    #plt.axvline(x=tmp_date)
    #plt.show()

    for e in metric_list:
        if e == "my_drawdown":
            df_comparations[e] = (my_drawdown(df_comparations.iloc[:,0]))*(-1)
        else:
            df_comparations[e] = df_comparations.iloc[:,0].expanding().apply(eval(e))
    df_comparations=df_comparations.ffill()
    df_comparations=df_comparations.fillna(0)
    df_comparations[(df_comparations==np.inf)|(df_comparations==-np.inf)]=0

    for e in df_comparations.iloc[:,1:].columns:
        if e != "my_drawdown" :
            df_comparations[e+"_std"] = df_comparations[e].loc[:real_money_date].std()
        else:
            df_comparations[e+"_perc"] = df_comparations[e].loc[:real_money_date].describe([.95])[-2]

    df_check=df_comparations.loc[real_money_date:]   

    if len(df_check)>0:
        for e in metric_list:
            df_check[e+"_check"]=1

        df_check["my_max_draw_down_check"] = np.where(df_check["my_max_draw_down"]>(df_check["my_max_draw_down"].iloc[0]+df_check["my_max_draw_down_std"]),0,df_check["my_max_draw_down_check"])

        df_check["my_ret_dd_check"] = np.where(df_check["my_ret_dd"]<(df_check["my_ret_dd"].iloc[0]-df_check["my_ret_dd_std"]),0,df_check["my_ret_dd_check"])

        df_check["my_sharpe_ratio_check"] = np.where(df_check["my_sharpe_ratio"]<(df_check["my_sharpe_ratio"].iloc[0]-df_check["my_sharpe_ratio_std"]),0,df_check["my_sharpe_ratio_check"])

        df_check["my_consec_loss_check"] = np.where(df_check["my_consec_loss"]>(df_check["my_consec_loss"].iloc[0]+df_check["my_consec_loss_std"]),0,df_check["my_consec_loss_check"])

        df_check["my_mean_loss_check"] = np.where(df_check["my_mean_loss"]>(df_check["my_mean_loss"].iloc[0]+df_check["my_mean_loss_std"]),0,df_check["my_mean_loss_check"])

        df_check["my_mean_win_check"] = np.where(df_check["my_mean_win"]<(df_check["my_mean_win"].iloc[0]-df_check["my_mean_win_std"]),0,df_check["my_mean_win_check"])

        df_check["my_avg_trade_check"] = np.where(df_check["my_avg_trade"]<(df_check["my_avg_trade"].iloc[0]-df_check["my_avg_trade_std"]),0,df_check["my_avg_trade_check"])

        df_check["my_drawdown_check"] = np.where(df_check["my_drawdown"]>df_check["my_drawdown_perc"],0,df_check["my_drawdown_check"])

        numero_condizioni = len(metric_list)

        df_check["total_check"] = df_check.iloc[:,-numero_condizioni:].sum(axis=1)
        df_check["total_check"][df_check["total_check"]<numero_condizioni]=0
        df_check["total_check"][df_check["total_check"]!=0]=1
        df_check["total_check"]=df_check["total_check"].shift(1 , fill_value=1)

        display(df_check)

        check_ON = df_check.iloc[np.where(df_check.total_check==0)]
        if len(check_ON)>0:
            stop_stratey=df_check.iloc[np.where(df_check.total_check==0)].index[0]
            strategy_fail.append(i)
            print(i,"FAIL !!!")
        else:
            stop_stratey=df_check.index[-1]
            strategy_ok.append(i)
            print(i,"OK")

        data_check = pd.concat([data_check,df_comparations.iloc[:,0].to_frame().loc[:stop_stratey]],axis=1).fillna(0)
    else:
        strategy_ok.append(i)
        print(i,"OK")
            
strategy_ok.sort()
strategy_fail.sort()

In [13]:
strategy_fail

['AUDCAD_Strategy_16',
 'AUDCHF_Strategy_20',
 'EURAUD_Strategy_19',
 'EURCHF_Strategy_3',
 'GBPCHF_Strategy_8']